# BBC021: Human MCF7 cells – compound-profiling experiment

In [ ]:
from scip_workflows.common import *

In [ ]:
import cellpose
from aicsimageio.readers.tiff_glob_reader import TiffGlobReader
from cellpose import models
from skimage.measure import regionprops


In [ ]:
data_dir = Path("/user/gent/420/vsc42015/vsc_data_vo/datasets/BBBC021")


In [ ]:
data_dir = Path("/home/maximl/scratch/data/vsc/datasets/BBBC021")


In [ ]:
batch = "Week5_28901"
name = "Week5_130707_F02_s3_w"


In [ ]:
def map_to_index(f):
    idx = re.match("^.*_w([0-9]).*$", f).group(1)
    m = {"1": 0, "2": 1, "4": 2}
    return pandas.Series(dict(S=0, T=0, C=m[idx], Z=0))


im = TiffGlobReader(
    glob_in=sorted(
        [str(p) for p in data_dir.glob("images/%s/%s*.tif" % (batch, name))]
    ),
    indexer=map_to_index,
    channel_names=["DAPI", "Actin", "Tubulin"],
)


In [ ]:
pixels = im.get_image_data("CXY")

fig, axes = plt.subplots(1, pixels.shape[0], dpi=150)
for ax, p in zip(axes, pixels):
    ax.imshow(p / p.reshape(1, -1).max(axis=1))
    ax.set_axis_off()


In [ ]:
model = models.Cellpose(gpu=True, model_type="cyto2")


In [ ]:
masks, flows, styles, diams = model.eval(pixels, diameter=None, channels=[2, 1])


In [ ]:
fig, ax = plt.subplots(1, 1, dpi=150)

ax.imshow((pixels / pixels.reshape(1, -1).max(axis=1))[1])
ax.set_axis_off()

ax.imshow(cellpose.plot.mask_rgb(masks), alpha=0.3)
for prop in regionprops(label_image=masks):
    ax.text(
        prop.centroid[1],
        prop.centroid[0],
        prop.label,
        fontsize=4,
        ha="center",
        va="center",
    )


In [ ]:
masks2, flows2, styles2, diams2 = model.eval(pixels, diameter=None, channels=[1, 1])


In [ ]:
fig, ax = plt.subplots(1, 1, dpi=150)

ax.imshow((pixels / pixels.reshape(1, -1).max(axis=1))[0])
ax.set_axis_off()

ax.imshow(cellpose.plot.mask_rgb(masks2), alpha=0.3)
for prop in regionprops(label_image=masks2):
    ax.text(
        prop.centroid[1],
        prop.centroid[0],
        prop.label,
        fontsize=4,
        ha="center",
        va="center",
    )


In [ ]:
from cellpose import plot

from scip.segmentation import cellpose


In [ ]:
import dask.bag
from dask.distributed import Client, LocalCluster


In [ ]:
c = Client(LocalCluster(n_workers=1))


In [ ]:
bag = dask.bag.from_sequence([dict(pixels=pixels)])


In [ ]:
events = bag.map_partitions(
    cellpose.segment_block, parent_channel_index=1, dapi_channel_index=0
).compute()


In [ ]:
mask = events[0]["mask"]


In [ ]:
plt.imshow(plot.mask_rgb(mask[0]))


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, dpi=300)
ax2.imshow(pixels[1])
ax1.imshow(pixels[1])
ax1.imshow(plot.mask_rgb(mask[1]), alpha=0.5)


In [ ]:
masks2, flows2, styles2, diams2 = model.eval(
    [pixels], diameter=None, channels=[2, 0], mask_threshold=0.2
)


In [ ]:
fig, ax = plt.subplots(1, 1, dpi=150)

ax.imshow((pixels / pixels.reshape(1, -1).max(axis=1))[1])
ax.set_axis_off()

ax.imshow(cellpose.plot.mask_rgb(masks2[0]), alpha=0.3)
for prop in regionprops(label_image=masks2[0]):
    ax.text(
        prop.centroid[1],
        prop.centroid[0],
        prop.label,
        fontsize=4,
        ha="center",
        va="center",
    )


In [ ]:
masks3, flows3, styles3, diams2 = model.eval(
    [pixels[0]], diameter=None, channels=[0, 0], mask_threshold=0.2
)


In [ ]:
fig, ax = plt.subplots(1, 1, dpi=150)

ax.imshow((pixels / pixels.reshape(1, -1).max(axis=1))[0])
ax.set_axis_off()

ax.imshow(cellpose.plot.mask_rgb(masks3[0]), alpha=0.3)
for prop in regionprops(label_image=masks3[0]):
    ax.text(
        prop.centroid[1],
        prop.centroid[0],
        prop.label,
        fontsize=4,
        ha="center",
        va="center",
    )
